<a href="https://colab.research.google.com/github/anjaleeDS/Sandbox/blob/master/covid_tweet_cleaner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



1. get tweets from "real users" 

cleaning:
- limit 10+ followers, 10+friends
- search for sources that have "bot" or "corona" in name or high frequency
sources that have repeated texts, high frequency writers
- remove user_names with "bot" or "covid19" or "corona" or "covid" or "Team"
or "Management" or "Society" or "Company"
- tweet texts that have duplicates (multiple dates with same text)
- user_names that contain ".com"

~70000 rows and 9 columns

3. sentiment analysis (kaggle task)
2. geographical distro (kaggle task)



In [31]:
# libs

import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gspread_dataframe import get_as_dataframe

In [32]:
pip install vaderSentiment

In [33]:
#sentiment analysis

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [34]:
#goog libs

# !pip install --upgrade gspread
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('ctweets1').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
dump = get_as_dataframe(worksheet=worksheet, header=0)

In [35]:
# dump.text.value_counts()
dump.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,#WearingAMaskIsLove,"Nairobi, Kenya",God First|Let's respectfully have honest convo...,########,1988.0,2596.0,195.0,0.0,########,"@JerotichSeii Hey there, I pray that you get b...",['COVID19'],Twitter for Android,0.0
1,#WearAMaskðŸ˜· #StayHome #SocialDistancing,The Twilight Zone,ðŸ‘ðŸ‘ My name is Talky Tina & you better be...,########,859.0,1950.0,80764.0,0.0,########,What's wrong with her lips? \n Was she stung b...,['COVID19'],Twitter for Android,0.0
2,#WearAMaskBear,"Cleveland Rocks, OH",Speak it into existence #BearMarket2020 #Stonk...,########,130.0,800.0,16001.0,0.0,########,Life is so hard right now for so many reasons....,NaN,Twitter for iPhone,0.0
3,#WearAMask ðŸ˜· Janet,NaN,"#Marketing Communications for #nonprofits, eng...",########,2500.0,2370.0,21715.0,0.0,########,Students and sports. \n What could go wrong?\n...,"['COVID19', 'NECOVID19']",Twitter for iPhone,0.0
4,#WearAMask #AmericaorTrump #hotspotUSA,NaN,#Voteblue: https://t.co/ryueFsXQTP #EndTheNigh...,########,1542.0,4986.0,26134.0,0.0,########,@realDonaldTrump Why not look into Canada And ...,NaN,Twitter Web App,0.0


In [36]:
#copy tweets into a separate column for cleaning
dump.tweet = dump.text
dump.insert(10, "tweet",dump.text, True) 



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [37]:
t_df = dump

In [38]:
# t_df = dump[['text', 'tweet']]
t_df = t_df.dropna(thresh=5) #dropping all rows that have 5+ NAs


In [39]:
t_df.tail(10)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,tweet,hashtags,source,is_retweet
10563,Bride Rose Sweeney,"Cleveland, OH",Youngest Democrat in the Ohio General Assembly...,########,1667.0,428.0,1010.0,1.0,########,Prayers to our Governor @MikeDeWine . Let this...,Prayers to our Governor @MikeDeWine . Let this...,"['COVID19', 'MasksOnOhio']",Twitter for iPhone,0.0
10564,Bridge Alliance,NaN,The leading mobile alliance for premier operat...,########,224.0,524.0,1361.0,0.0,########,It's #nationalday here in Singapore! ðŸ‡¸ðŸ‡¬ ...,It's #nationalday here in Singapore! ðŸ‡¸ðŸ‡¬ ...,['nationalday'],Twitter Web App,0.0
10565,Bridge Consulting,"Beijing, China","A mission-driven, issue-based communications c...",########,45.0,41.0,19.0,0.0,########,â€œWe must remember that most people are still...,â€œWe must remember that most people are still...,['COVID19'],Twitter Web App,0.0
10566,BRIDGEi2i,"Bangalore, Fremont",Leading AI Solutions company. Driving business...,########,6433.0,3583.0,2738.0,0.0,########,"""Digital adoption is going to intensify, &amp;...","""Digital adoption is going to intensify, &amp;...",['AI'],HubSpot,0.0
10567,Bridget BarclaySudol,NaN,Only follow me if you like aimless walks and a...,########,14.0,28.0,619.0,0.0,########,Care package in the time of a pandemic: just r...,Care package in the time of a pandemic: just r...,NaN,Twitter for iPhone,0.0
10568,Bridget Goulding,"Cork, Ireland",Mid-Cork: Nature Watcher: concerned++ re loss ...,########,806.0,1487.0,74660.0,0.0,########,Please everybody who needs to contact your fam...,Please everybody who needs to contact your fam...,"['respiratorysymptoms', 'COVID19']",Twitter for iPhone,0.0
10569,Bridget Welsh,malaysia,"Educator, Scholar and Political Analyst, Focus...",########,789.0,471.0,514.0,0.0,########,#Malaysia's PM #Muhyiddin at a national flag c...,#Malaysia's PM #Muhyiddin at a national flag c...,"['Malaysia', 'Muhyiddin', 'COVID19']",Twitter for Android,0.0
10570,"Bridgette M. Brawner, PhD, MDiv, APRN","Philadelphia, PA",Assoc. Prof. at @PennNursing; Chair of @anamfp...,########,1172.0,396.0,2646.0,0.0,########,"@DearPandemic, can you answer some timeline qu...","@DearPandemic, can you answer some timeline qu...",['COVID19'],Twitter for Android,0.0
10571,Brie Blackstone,"Federal Way, WA","girl who meanders, sings, writes, loves cultur...",########,159.0,1714.0,3935.0,0.0,########,@Hertz wow this is your gold standard of clean...,@Hertz wow this is your gold standard of clean...,"['pandemic', 'attitude']",Twitter for Android,0.0
10572,"Brief19, your daily roundup of SARS-CoV-2.","Everywhere, unfortunately",A daily roundup of SARS-CoV-2 / covid-19 resea...,########,14140.0,339.0,531.0,0.0,########,Antibody Levels Fall After SARS-CoV-2 Infectio...,Antibody Levels Fall After SARS-CoV-2 Infectio...,NaN,Twitter Web App,0.0


In [40]:
t_df.tweet =t_df.tweet.str.replace(r'[^a-zA-Z0-9.\@,\' ]?', r'')
t_df.tail(10)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,tweet,hashtags,source,is_retweet
10563,Bride Rose Sweeney,"Cleveland, OH",Youngest Democrat in the Ohio General Assembly...,########,1667.0,428.0,1010.0,1.0,########,Prayers to our Governor @MikeDeWine . Let this...,Prayers to our Governor @MikeDeWine . Let this...,"['COVID19', 'MasksOnOhio']",Twitter for iPhone,0.0
10564,Bridge Alliance,NaN,The leading mobile alliance for premier operat...,########,224.0,524.0,1361.0,0.0,########,It's #nationalday here in Singapore! ðŸ‡¸ðŸ‡¬ ...,It's nationalday here in Singapore Our member...,['nationalday'],Twitter Web App,0.0
10565,Bridge Consulting,"Beijing, China","A mission-driven, issue-based communications c...",########,45.0,41.0,19.0,0.0,########,â€œWe must remember that most people are still...,We must remember that most people are still su...,['COVID19'],Twitter Web App,0.0
10566,BRIDGEi2i,"Bangalore, Fremont",Leading AI Solutions company. Driving business...,########,6433.0,3583.0,2738.0,0.0,########,"""Digital adoption is going to intensify, &amp;...","Digital adoption is going to intensify, amp in...",['AI'],HubSpot,0.0
10567,Bridget BarclaySudol,NaN,Only follow me if you like aimless walks and a...,########,14.0,28.0,619.0,0.0,########,Care package in the time of a pandemic: just r...,Care package in the time of a pandemic just re...,NaN,Twitter for iPhone,0.0
10568,Bridget Goulding,"Cork, Ireland",Mid-Cork: Nature Watcher: concerned++ re loss ...,########,806.0,1487.0,74660.0,0.0,########,Please everybody who needs to contact your fam...,Please everybody who needs to contact your fam...,"['respiratorysymptoms', 'COVID19']",Twitter for iPhone,0.0
10569,Bridget Welsh,malaysia,"Educator, Scholar and Political Analyst, Focus...",########,789.0,471.0,514.0,0.0,########,#Malaysia's PM #Muhyiddin at a national flag c...,Malaysia's PM Muhyiddin at a national flag cer...,"['Malaysia', 'Muhyiddin', 'COVID19']",Twitter for Android,0.0
10570,"Bridgette M. Brawner, PhD, MDiv, APRN","Philadelphia, PA",Assoc. Prof. at @PennNursing; Chair of @anamfp...,########,1172.0,396.0,2646.0,0.0,########,"@DearPandemic, can you answer some timeline qu...","@DearPandemic, can you answer some timeline qu...",['COVID19'],Twitter for Android,0.0
10571,Brie Blackstone,"Federal Way, WA","girl who meanders, sings, writes, loves cultur...",########,159.0,1714.0,3935.0,0.0,########,@Hertz wow this is your gold standard of clean...,@Hertz wow this is your gold standard of clean...,"['pandemic', 'attitude']",Twitter for Android,0.0
10572,"Brief19, your daily roundup of SARS-CoV-2.","Everywhere, unfortunately",A daily roundup of SARS-CoV-2 / covid-19 resea...,########,14140.0,339.0,531.0,0.0,########,Antibody Levels Fall After SARS-CoV-2 Infectio...,Antibody Levels Fall After SARSCoV2 Infection....,NaN,Twitter Web App,0.0


EidalAdha knocks the door, it is crucial to reiterate that we are living in unusual times where normalcy has bec httpst.coZzmzFgPQPJ {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
The Senator representing Oyo Central stated that the decision to invite the ministers and the @NNPCgroup GMD arrive httpst.coFa5C3EuJ9Z {'neg': 0.0, 'neu': 0.918, 'pos': 0.082, 'compound': 0.1531}
Members of President Trumps New Jersey golf club jeered a reporter who asked why they weren't complying with a req httpst.conrcXLChC87 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Calling someone a germaphobe used to mean something. Now we have been told this is the new normal.   Not for me COVID19 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
URGENT Eleven Illinois counties are currently at a warning level for COVID19  Cass, Gallatin, Jackson, Jo Davies httpst.co0L5msk2Kao {'neg': 0.115, 'neu': 0.764, 'pos': 0.121, 'compound': 0.0343}
The Illinois Department of Public Health is reporting 1,467 new cases and 14 deaths today. New Totals 181,943 case httpst.covV1Ma6h6vT {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [41]:
# t_df = t_df.drop(axis=1, labels='sentiments')
# t_df

In [42]:
sents = []

analyzer = SentimentIntensityAnalyzer()
for t in t_df.tweet:
  vs = analyzer.polarity_scores(t)
  # print("{:-<65} {}".format(t, str(vs)))
  # print(str(vs))
  sents.append(str(vs))

t_df.insert(14, "sentiments",sents, True) 
t_df


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,tweet,hashtags,source,is_retweet,sentiments
0,#WearingAMaskIsLove,"Nairobi, Kenya",God First|Let's respectfully have honest convo...,########,1988.0,2596.0,195.0,0.0,########,"@JerotichSeii Hey there, I pray that you get b...","@JerotichSeii Hey there, I pray that you get b...",['COVID19'],Twitter for Android,0.0,"{'neg': 0.0, 'neu': 0.766, 'pos': 0.234, 'comp..."
1,#WearAMaskðŸ˜· #StayHome #SocialDistancing,The Twilight Zone,ðŸ‘ðŸ‘ My name is Talky Tina & you better be...,########,859.0,1950.0,80764.0,0.0,########,What's wrong with her lips? \n Was she stung b...,What's wrong with her lips Was she stung by a...,['COVID19'],Twitter for Android,0.0,"{'neg': 0.205, 'neu': 0.795, 'pos': 0.0, 'comp..."
2,#WearAMaskBear,"Cleveland Rocks, OH",Speak it into existence #BearMarket2020 #Stonk...,########,130.0,800.0,16001.0,0.0,########,Life is so hard right now for so many reasons....,Life is so hard right now for so many reasons....,NaN,Twitter for iPhone,0.0,"{'neg': 0.056, 'neu': 0.854, 'pos': 0.091, 'co..."
3,#WearAMask ðŸ˜· Janet,NaN,"#Marketing Communications for #nonprofits, eng...",########,2500.0,2370.0,21715.0,0.0,########,Students and sports. \n What could go wrong?\n...,Students and sports. What could go wrong COVI...,"['COVID19', 'NECOVID19']",Twitter for iPhone,0.0,"{'neg': 0.256, 'neu': 0.744, 'pos': 0.0, 'comp..."
4,#WearAMask #AmericaorTrump #hotspotUSA,NaN,#Voteblue: https://t.co/ryueFsXQTP #EndTheNigh...,########,1542.0,4986.0,26134.0,0.0,########,@realDonaldTrump Why not look into Canada And ...,@realDonaldTrump Why not look into Canada And ...,NaN,Twitter Web App,0.0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10568,Bridget Goulding,"Cork, Ireland",Mid-Cork: Nature Watcher: concerned++ re loss ...,########,806.0,1487.0,74660.0,0.0,########,Please everybody who needs to contact your fam...,Please everybody who needs to contact your fam...,"['respiratorysymptoms', 'COVID19']",Twitter for iPhone,0.0,"{'neg': 0.0, 'neu': 0.892, 'pos': 0.108, 'comp..."
10569,Bridget Welsh,malaysia,"Educator, Scholar and Political Analyst, Focus...",########,789.0,471.0,514.0,0.0,########,#Malaysia's PM #Muhyiddin at a national flag c...,Malaysia's PM Muhyiddin at a national flag cer...,"['Malaysia', 'Muhyiddin', 'COVID19']",Twitter for Android,0.0,"{'neg': 0.12, 'neu': 0.737, 'pos': 0.143, 'com..."
10570,"Bridgette M. Brawner, PhD, MDiv, APRN","Philadelphia, PA",Assoc. Prof. at @PennNursing; Chair of @anamfp...,########,1172.0,396.0,2646.0,0.0,########,"@DearPandemic, can you answer some timeline qu...","@DearPandemic, can you answer some timeline qu...",['COVID19'],Twitter for Android,0.0,"{'neg': 0.0, 'neu': 0.856, 'pos': 0.144, 'comp..."
10571,Brie Blackstone,"Federal Way, WA","girl who meanders, sings, writes, loves cultur...",########,159.0,1714.0,3935.0,0.0,########,@Hertz wow this is your gold standard of clean...,@Hertz wow this is your gold standard of clean...,"['pandemic', 'attitude']",Twitter for Android,0.0,"{'neg': 0.0, 'neu': 0.764, 'pos': 0.236, 'comp..."


In [43]:
# t_df.tweet =t_df.tweet.str.replace(r'[^a-zA-Z0-9.\@,\' ]?', r'')
# t_df.tail(10)

In [44]:
# t_df.sentiments.str.replace(r'[a-z:{}\' ]?', r'')

sscore = t_df.sentiments.str.replace(r'[a-z:{}\' ]?', r'')
sscore = sscore.str.split(pat=',',expand=True)
sscore.columns = ['neg', 'neu', 'pos', 'compound']

t_df.insert(15, "neg",sscore.neg, True) 
t_df.insert(16, "neu",sscore.neu, True) 
t_df.insert(17, "pos",sscore.pos, True) 
t_df.insert(18, "compound",sscore.compound, True) 


# t_df.sentiments.str.replace(r'[^a-zA-Z0-9.\@,\' ]?', r'')
# t_df.sentiments

In [55]:
t_df.iloc[:, [15,16,17,18]] = t_df.iloc[:,[15,16,17,18]].astype('float64')
t_df.dtypes

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


user_name            object
user_location        object
user_description     object
user_created         object
user_followers      float64
user_friends        float64
user_favourites     float64
user_verified       float64
date                 object
text                 object
tweet                object
hashtags             object
source               object
is_retweet          float64
sentiments           object
neg                 float64
neu                 float64
pos                 float64
compound            float64
dtype: object

In [68]:

t_df.tweet[(t_df.neu > 0.8)]

2        Life is so hard right now for so many reasons....
4        @realDonaldTrump Why not look into Canada And ...
5        53K new coronavirus cases today @realDonaldTru...
8        @TeamTrump @SebRougemont What public polling p...
10       Oh the irony Gohmert COVID19 Covidiots httpst....
                               ...                        
10565    We must remember that most people are still su...
10566    Digital adoption is going to intensify, amp in...
10568    Please everybody who needs to contact your fam...
10570    @DearPandemic, can you answer some timeline qu...
10572    Antibody Levels Fall After SARSCoV2 Infection....
Name: tweet, Length: 6802, dtype: object

#TF-IDF for recommendations

Recommend who to follow if you want generally negative, positive or neutral tweets

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
ihotel = ihotel.fillna('')

tfidf_matrix = tfidf.fit_transform(ihotel.clean_desc)


In [ ]:
tfidf_matrix.shape

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim.shape
# (cosine_sim<0).sum()

In [ ]:
indices = pd.Series(ihotel.index, index=ihotel.property_name).drop_duplicates()

In [ ]:
indices

In [ ]:
indices.value_counts().max()

In [ ]:
idx = indices['TRH Pipalkoti']
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:11]
movie_indices =  [i[0] for i in sim_scores]
movie_indices

In [ ]:
idx

In [ ]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return ihotel.clean_desc.iloc[movie_indices] 

In [ ]:
get_recommendations('TRH Pipalkoti')